In [2]:
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..')))

from SimulateDay import *


def calculate_percentage(fee, SalePrice, stock):
    if SalePrice == 0:
        return 0
    percentage = (fee / SalePrice) * 100
    portfolio = pd.read_csv('portfolio.csv')
    
    # Update the SalePrice for the given stock on the most recent day
    portfolio.loc[(portfolio['Stock Name'] == stock) & (portfolio['Day'] == portfolio['Day'].max()), 'SalePrice'] = SalePrice - fee
    
    # Write to the log file in append mode
    with open('Log.txt', 'a') as file:
        file.write(f"{stock}, sold at {SalePrice - fee}, fee: {fee}, percentage: {percentage}\n")
    
    # Print the log message
    print(f"{stock}, sold at {SalePrice - fee}, fee: {fee}, percentage: {percentage}")
    
    return portfolio

# Example usage
fee = 0.02
SalePrice = 124.13
stock = "AMZN"
Sale = calculate_percentage(fee, SalePrice, stock).fillna(0)

AMZN, sold at 124.11, fee: 0.02, percentage: 0.016112140497865142


In [6]:
def execute_buy(stock_name, purchase_price):
    portfolio = pd.read_csv('portfolio.csv')
    stock = portfolio[(portfolio['Stock Name'] == stock_name) & (portfolio['Day'] == portfolio['Day'].max())]
    
    if len(stock) == 0:
        # Fetch stock data
        stock_data = get_stock_data(stock_name)
        stock_price = stock_data['Close'].iloc[-1]
        
        # Calculate the number of shares that can be bought
        shares = purchase_price / stock_price
        
        # Append the new transaction to the portfolio
        portfolio = portfolio.append({
            'Stock Name': stock_name,
            'Day': portfolio['Day'].max() + 1,
            'Action': 'Buy',
            'Stock Price': stock_price,
            'Cash': purchase_price - (shares * stock_price),
            'Shares Held': shares,
            'Portfolio Value': shares * stock_price,
            'Date': stock_data['Date'].iloc[-1],
            'Actual Sell': 0
        }, ignore_index=True)
    else:
        stock_price = stock['Stock Price'].values[0]
        shares_to_buy = purchase_price / stock_price
        shares = stock['Shares Held'].values[0] + shares_to_buy
        portfolio.loc[(portfolio['Stock Name'] == stock_name) & (portfolio['Day'] == portfolio['Day'].max()), 'Shares Held'] += shares_to_buy 
        portfolio.loc[(portfolio['Stock Name'] == stock_name) & (portfolio['Day'] == portfolio['Day'].max()), 'Actual Buy'] = True
        portfolio.loc[(portfolio['Stock Name'] == stock_name) & (portfolio['Day'] == portfolio['Day'].max()), 'Portfolio Value'] = (shares * stock_price)

    portfolio.to_csv('portfolio.csv', index=False)
    
    return portfolio

purchase_price = 5
stock_name = ""
portfolio = execute_buy(stock_name, purchase_price)
portfolio.tail(7)

,Stock Name,Day,Action,Stock Price,Cash,Shares Held,Portfolio Value,Date,Actual Sell,Actual Buy
49,AAPL,7,Hold,222.464996,0,0.331895,73.835047,2024-11-05,0.0,NaN
50,XOM,7,Buy,118.239998,0,0.896151,105.960887,2024-11-05,0.0,True
51,AMZN,7,Buy,198.789993,0,0.771653,153.396832,2024-11-05,0.0,True
52,LYV,7,Buy,117.559998,0,0.591747,69.565730,2024-11-05,0.0,True
53,INTC,7,Buy,23.200001,0,3.211000,74.495205,2024-11-05,0.0,True
54,DNMR,7,Hold,0.318400,0,1.820000,0.579488,2024-11-05,0.0,NaN
55,SPCE,7,Hold,6.585000,0,0.246548,1.623521,2024-11-05,0.0,NaN


In [7]:
portfolio.tail(7)['Portfolio Value'].sum()  

479.4567095522166

In [18]:
20/22.205000

0.900698040981761